In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
from src.network import Network_custom
import plotly.io as pio
%matplotlib qt
pio.renderers.default = 'browser'

### Open the network model

In [ ]:
BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"
# model_name  = 'Validation_structure_1'
# model_name = 'VS_ring0.5_connectors0.8_center0.2_as1.5_asr1.5_s110'
# model_name = 'unit_cell_loop_size_q_(1.5, 2, 0.7)'
# model_name = 'unit_cell_loop_size_q_(1.5, 2, 0.7)_10'
# model_name = 'unit_cell_even_q_0.6'
# model_name = 'unit_cell_loop_size_q_(4, 3, 0.6)_10'
model_name = "unit_cell_loop_size_q_(13, 14, 0.35)_10"
try:
    net = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '.pkl'))
    rotate45 = False
except:
    net = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))
    rotate45 = True
net.net_plot(color=True, elables = False, vlabels = True)

approx_shift = np.array([1.8 * np.max(net.vertices[:, 0]), 1.5 * np.max(net.vertices[:, 1]), 0.0])
approx_scale = 20

vertices_prescaled = net.vertices.copy()
vertices_prescaled += approx_shift
vertices_prescaled *= approx_scale

picture_number = 0
model_name_im = model_name + f"_{picture_number}"

# try:
image_name = model_name_im + ".jpg"
image_path = os.path.join(BYU_UW_root, 'measuerement images', image_name)
original_image = plt.imread(image_path)
# except FileNotFoundError:
#     image_name = model_name_im + ".nef"
#     image_path = os.path.join(BYU_UW_root, 'measuerement images', image_name)
#     original_image = plt.imread(image_path)
height, width, _ = original_image.shape

# Note. I found that undistorting the image does not get better results. The distortion coefficients also contain values that are unrealistic. I am not sure why

# mtx = np.array([[2.68908731e+04, 0.00000000e+00, 2.26405663e+03],
#  [0.00000000e+00, 2.68576281e+04, 2.94945084e+03],
#  [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# dist = np.array([[ 0.8429211,  -0.30036892, -0.00690671,  0.01086315,  0.        ]])

# undistorted_image = cv2.undistort(original_image, mtx, dist)

# original_image = undistorted_image

Flip and rotate if necessary

In [ ]:
from matplotlib.widgets import Button
# === State trackers ===
rotation_state = [0]       # in degrees, e.g. 0, 90, 180, 270
flip_ud_state = [False]    # vertical flip state
image_displayed = [None]   # store current image
ax = [None]                # access axis in handlers

def apply_transform():
    """Apply rotation and flip to the original image and return the transformed image."""
    img = original_image.copy()
    # Flip up-down if needed
    if flip_ud_state[0]:
        img = np.flipud(img)
    # Rotate based on rotation_state
    k = rotation_state[0] // 90
    img = np.rot90(img, k=k)
    return img

def update_display():
    ax[0].clear()
    transformed = apply_transform()
    ax[0].imshow(transformed)
    ax[0].axis("off")
    ax[0].set_title(f"Rot: {rotation_state[0]}°, FlipUD: {flip_ud_state[0]}")
    ax[0].figure.canvas.draw()

def rotate_plus(event):
    rotation_state[0] = (rotation_state[0] + 90) % 360
    update_display()

def rotate_minus(event):
    rotation_state[0] = (rotation_state[0] - 90) % 360
    update_display()

def flip_ud(event):
    flip_ud_state[0] = not flip_ud_state[0]
    update_display()

def on_close(event):
    global original_image
    original_image = apply_transform()
    height, width, _ = original_image.shape
    print("Image window closed. `original_image` updated with final transformation.")

# === Plot image ===
fig, main_ax = plt.subplots(figsize=(12, 12))
ax[0] = main_ax
fig.canvas.manager.set_window_title("Image Viewer")
image_displayed[0] = main_ax.imshow(apply_transform())
main_ax.axis("off")
main_ax.set_title(f"Rot: {rotation_state[0]}°, FlipUD: {flip_ud_state[0]}")
# Connect close event
fig.canvas.mpl_connect("close_event", on_close)

# === Buttons ===
button_fig, _ = plt.subplots(figsize=(3, 2))
button_fig.canvas.manager.set_window_title("Controls")

ax_rotp = plt.axes([0.1, 0.6, 0.8, 0.25])
ax_rotm = plt.axes([0.1, 0.35, 0.8, 0.25])
ax_flip = plt.axes([0.1, 0.1, 0.8, 0.25])

btn_rotp = Button(ax_rotp, "+90°")
btn_rotm = Button(ax_rotm, "-90°")
btn_flip = Button(ax_flip, "Flip Up/Down")

btn_rotp.on_clicked(rotate_plus)
btn_rotm.on_clicked(rotate_minus)
btn_flip.on_clicked(flip_ud)

plt.show()


### Load an image of the frame and click on the markers

In [ ]:
xlim_init, ylim_init = (0, width), (height, 0)
def on_click(event):
    """Reset view when clicking anywhere in the figure."""
    if event.button == 1:  # Left mouse button
        points.append((event.xdata, event.ydata))
        ax.plot(event.xdata, event.ydata, 'r.', label='real vertices')
        ax.set_xlim(xlim_init)
        ax.set_ylim(ylim_init)
        ax.figure.canvas.draw()

def on_scroll(event):
    """Zoom in/out at mouse position using the scroll wheel."""
    scale_factor = 2.0  # Zoom speed
    xlim, ylim = ax.get_xlim(), ax.get_ylim()

    if event.step > 0:  # Zoom in
        xlim_new = (event.xdata - (event.xdata - xlim[0]) / scale_factor,
                    event.xdata + (xlim[1] - event.xdata) / scale_factor)
        ylim_new = (event.ydata - (event.ydata - ylim[0]) / scale_factor,
                    event.ydata + (ylim[1] - event.ydata) / scale_factor)
    else:  # Zoom out
        xlim_new = (event.xdata - (event.xdata - xlim[0]) * scale_factor,
                    event.xdata + (xlim[1] - event.xdata) * scale_factor)
        ylim_new = (event.ydata - (event.ydata - ylim[0]) * scale_factor,
                    event.ydata + (ylim[1] - event.ydata) * scale_factor)

    ax.set_xlim(xlim_new)
    ax.set_ylim(ylim_new)
    fig.canvas.draw()

def on_motion(event):
    """Update ring cursor position."""
    if event.xdata is not None and event.ydata is not None:
        circle_outer.set_center((event.xdata, event.ydata))
        circle_inner.set_center((event.xdata, event.ydata))
        fig.canvas.draw_idle()

def on_motion2(event):
    """Update ring cursor position."""
    if event.xdata is not None and event.ydata is not None:
        circle.set_center((event.xdata, event.ydata))
        circle.set_center((event.xdata, event.ydata))
        fig.canvas.draw_idle()

def on_close(event):
    """Callback function to stop the event loop when the figure is closed."""
    plt.close()

In [ ]:
points = []
# Display image
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(original_image)
# ax.set_xlim(xlim_init)
# ax.set_ylim(ylim_init)
ax.axis("off")

# Add quadrant markers
ax.text(width/2, height*.9, '1', color='r')
ax.text(width*.9, height/2, '2', color='r')
ax.text(width/2, height*.1, '3', color='r')
ax.text(width*.1, height/2, '4', color='r')
ax.set_title("Click on markers (in order). Zoom with scroll wheel.")

# Create ring cursor
circle_outer = plt.Circle((0, 0), radius=40, color='r', fill=False, lw=2)
circle_inner = plt.Circle((0, 0), radius=20, color='r', fill=False, lw=2)
ax.add_patch(circle_outer)
ax.add_patch(circle_inner)


# Connect events
fig.canvas.mpl_connect("button_press_event", on_click)
fig.canvas.mpl_connect("scroll_event", on_scroll)
fig.canvas.mpl_connect("motion_notify_event", on_motion)

plt.show()

# directly:
# marker_points_px = np.array([(3108.1745194618484, 3402.5666577771403),
#  (4642.920922553315, 1912.6263830304224),
#  (3143.682438629892, 391.1770490488246),
#  (1615.726921448857, 1880.5276926742765)])

#after undistort:
# marker_points_px = np.array([(3106.461517421706, 3403.8021503327177),
#  (4616.727234909741, 1924.5659905629545),
#  (3132.4174313825924, 420.92588907903246),
#  (1616.1038240720675, 1882.460629651013)])

# marker_points_px

Use this points if you don't want to reassign the points everytime you run the script

### Calibrate the camera angle with the known coordinates of the markers.
This will break if the order of the coordinates are misaligned. Also notice that the y-axis is inverted when working with images, ensure your know cordinate list is set up accordingly

In [ ]:
marker_points_px = np.array(points)
distance_between_markers_mm = 153.76#155.563491 #148.49 # mm 155.669 # 
distance_between_markers_px0 = np.linalg.norm(marker_points_px[0] - marker_points_px[2])
distance_between_markers_px1 = np.linalg.norm(marker_points_px[1] - marker_points_px[3])
distance_between_markers_px = (distance_between_markers_px0 + distance_between_markers_px1) / 2
points_real = np.array([(0, -distance_between_markers_mm/2), (distance_between_markers_mm/2, 0), (0, distance_between_markers_mm/2), (-distance_between_markers_mm/2, 0)])
points_real += approx_shift[:2]
points_real *= approx_scale

homography_px_to_real, _ = cv2.findHomography(marker_points_px, points_real, method=cv2.RANSAC)
homography_real_to_px, _ = cv2.findHomography(points_real, marker_points_px, method=cv2.RANSAC)
aligned_image = cv2.warpPerspective(original_image, homography_px_to_real, (width, height))

points_real_validate = net.apply_homography(marker_points_px, homography_px_to_real)
vertices_px = net.apply_homography(vertices_prescaled, homography_real_to_px)

fig, ax = plt.subplots(2, 1, figsize=(8, 13), sharex=True, sharey=True)
ax[0].imshow(original_image)
ax[0].plot(marker_points_px[:, 0], marker_points_px[:, 1], 'r-*', markersize=10)
ax[0].plot(marker_points_px[0, 0], marker_points_px[0, 1], 'bo', markersize=10)
ax[0] = net.net_plot_mat(ax[0], vlabels=True, fp=True, vertices_c = vertices_px)
ax[0].set_title('Unaligned Image')
ax[0].axis('off')
ax[1].imshow(aligned_image)
ax[1] = net.net_plot_mat(ax[1], vlabels=True, fp=True, vertices_c = vertices_prescaled)
# ax[1].plot(points_aligned[:, 0], points_aligned[:, 1], 'r-*', markersize=10, label='known points')
ax[1].plot(points_real_validate[:, 0], points_real_validate[:, 1], 'g.', markersize=10, label='known points - validate')
ax[1].plot(points_real_validate[0, 0], points_real_validate[0, 1], 'bo', markersize=10, label='known points - validate')
ax[1].set_title('Orthogonal Image')
ax[1].axis('off')
plt.title("Both networks should approximately match the picture")
plt.legend()

points = []
fig, ax = plt.subplots(figsize=(15, 10))
ax.axis('off')
ax.set_aspect('equal')
ax.imshow(original_image)
ax = net.net_plot_mat(ax, vlabels=True, fp=True, vertices_c = vertices_px)

circle = plt.Circle((0, 0), radius=99, color='r', fill=False, lw=2) #4.88 mm
ax.add_patch(circle)

fig.canvas.mpl_connect('button_press_event', on_click)
fig.canvas.mpl_connect('scroll_event', on_scroll)
fig.canvas.mpl_connect("motion_notify_event", on_motion2)
fig.canvas.mpl_connect('close_event', on_close)
ax.set_title('Click on the vertices of the network (in order). (Scroll to zoom)')
plt.show()

In [ ]:
points_px = np.array(points)
points_mm = net.apply_homography(points_px, homography_px_to_real)
points_mm, vertices_prescaled

points_mm_unscaled = points_mm.copy()
points_mm_unscaled /= approx_scale
points_mm_unscaled -= approx_shift[:2]


file_path = os.path.join(BYU_UW_root, 'Tensile Testing', 'Avg_Stress_Strain_Overture_TPU_1mm2.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)
strain_to_stress = net.material_model(stress_data, strain_data, interpolation_kind = 'cubic')
TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive
A = [TPU_nl['A']]*len(net.edges)

vertices_equilibrium = net.vertices.copy()
vertices_equilibrium[net.fixed,:2] = points_mm_unscaled[net.fixed]

vertices_equilibrium_unscaled, l1_equilibrium, f = net.find_equilibrium(vertices_equilibrium, A, strain_to_stress)
vertices_equilibrium = vertices_equilibrium_unscaled.copy()
vertices_equilibrium += approx_shift
vertices_equilibrium *= approx_scale
vertices_equilibrium_px = net.apply_homography(vertices_equilibrium, homography_real_to_px)

fig, ax = plt.subplots(figsize=(15, 10))
ax.axis('off')
ax.set_aspect('equal')
ax.imshow(original_image)
ax = net.net_plot_mat(ax, vlabels=True, fp=True, vertices_c=vertices_equilibrium_px)
ax.plot(points_px[:, 0], points_px[:, 1], 'ko', markersize=3, label='Real vertices')
ax.plot(vertices_px[net.fixed,0], vertices_px[net.fixed,1], 'b*', markersize=5, label='Fixed points - designed')
ax.set_xlim(np.min(vertices_px[net.fixed,0]) - 100, np.max(vertices_px[net.fixed,0])+ 100)
ax.set_ylim(np.max(vertices_px[net.fixed,1])+ 100, np.min(vertices_px[net.fixed,1])- 100)
ax.legend(fontsize=12)
plt.show()
plt.savefig(os.path.join(BYU_UW_root, 'images','validation_images', model_name_im + '_val.png'), dpi=300, bbox_inches='tight')

In [ ]:
# Save the calibration results
calibration_results = {
    'model_name': model_name,
    'model_name_im': model_name_im,
    # 'camera_matrix': mtx,
    # 'distortion_coefficients': dist,
    'homography_matrix': homography_real_to_px,
    'homography_matrix_inv': homography_px_to_real,
    'real_vertices': points_mm_unscaled,
    'equilibrium_vertices': vertices_equilibrium_unscaled,
    'vertices': net.vertices,
}
np.save(os.path.join(BYU_UW_root, 'calibration_results', model_name_im + '_cr.npy'), calibration_results, allow_pickle=True)

model_name_im

In [2]:
BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"
fig, ax = plt.subplots(1, 3, figsize=(15, 10), sharey=True)
ax[0].set_xlabel('Force (N)')
ax[1].set_xlabel('Curvature (1/m)')
ax[2].set_xlabel('Initial length (m)')
ax[0].set_ylabel('Relative error (%)')

calibration_results_files = glob.glob(os.path.join(BYU_UW_root, 'calibration_results', '*_cr.npy'))

for file in calibration_results_files:
    try:
        data = np.load(file, allow_pickle=True).item()
    except:
        print(f'for some reason {file} stopped working')
        continue
    model_name_im = data['model_name_im']
    model_name = data['model_name']
    real_vertices = data['real_vertices']
    vertices_equilibrium = data['equilibrium_vertices']

    print(model_name)
    try:
        net2 = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '.pkl'))
    except:
        net2 = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl')) 

    f = net2.f
    R = net2.R
    kappa = 1/R
    l0 = net2.l0
    error_edge = []
    for edge_i, edge in enumerate(net2.edges):
        coor0, coor1 = real_vertices[edge[0]], real_vertices[edge[1]]
        l1m = np.linalg.norm(coor0[:2] - coor1[:2])

        coor2, coor3 = vertices_equilibrium[edge[0]], vertices_equilibrium[edge[1]]
        l1_equilibrium = np.linalg.norm(coor2[:2] - coor3[:2])

        error_edge.append(np.abs(l1m - l1_equilibrium) / l1_equilibrium)
    edge_error = np.array(error_edge) * 100 # percentage error
    arg_f_max = np.argmax(f)
    arg_kappa_max = np.argmax(kappa)
    
    ax[0].plot(f[arg_f_max], error_edge[arg_f_max], 'o', label=model_name_im)
    ax[1].plot(kappa[arg_kappa_max], error_edge[arg_kappa_max], 'o', label=model_name_im)
    ax[2].plot(l0, error_edge, 'o', label=model_name_im)

ax[0].legend()
ax[0].set_title('Force vs Relative Error')
ax[1].set_title('Curvature vs Relative Error')
ax[2].set_title('Initial Length vs Relative Error')
plt.tight_layout()
plt.show()

    

unit_cell_loop_size_q_(13, 14, 0.35)_0.79
unit_cell_loop_size_q_(13, 14, 0.35)_0.86
unit_cell_even_q_0.6_10
unit_cell_even_q_0.8_10
unit_cell_even_q_0.35_10
unit_cell_even_q_1.0_10
unit_cell_even_q_1.0_0.65
unit_cell_even_q_1.0_0.71
unit_cell_even_q_1.0_0.78
unit_cell_even_q_1.0_0.84
unit_cell_even_q_1.0_0.90
unit_cell_loop_size_q_(4, 3, 0.6)_10
unit_cell_loop_size_q_(6, 5, 0.4)_10
unit_cell_loop_size_q_(13, 14, 0.35)_0.65
unit_cell_loop_size_q_(13, 14, 0.35)_0.72
unit_cell_loop_size_q_(13, 14, 0.35)_0.92
unit_cell_loop_size_q_(13, 14, 0.35)_10
